# Interpretibility

Indentifies important wavenumbers for density, crystallinity and SCB prediction, performs SHAP visualization, and looks at correlations between intensities at different wavenumbers

# Dependencies

In [ ]:
import os
import glob
import numpy as np
from numpy.dtypes import StringDType
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import shap
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
import scipy
import copy
from scipy.stats import pearsonr

# Functions

## Get data

In [ ]:
def getdata(foldername, fold_idx=0, base_path="../ModelData"):

    folder_path = os.path.join(base_path, foldername)

    wavexTrain = np.loadtxt(
        os.path.join(folder_path, "X_train_" + str(fold_idx) + ".csv"), delimiter=","
    )
    wavecoeffsTrain = np.loadtxt(
        os.path.join(folder_path, "PLSR_coef_" + str(fold_idx) + ".csv"), delimiter=","
    )
    ypredTrain = np.loadtxt(
        os.path.join(folder_path, "ypred_PLSR_train_" + str(fold_idx) + ".csv"),
        delimiter=",",
    )
    yactTrain = np.loadtxt(
        os.path.join(folder_path, "y_train_" + str(fold_idx) + ".csv"), delimiter=","
    )
    interceptTrain = np.loadtxt(
        os.path.join(folder_path, "PLSR_intercept_" + str(fold_idx) + ".csv"),
        delimiter=",",
    )

    wavexTest = np.loadtxt(
        os.path.join(folder_path, "X_test_" + str(fold_idx) + ".csv"), delimiter=","
    )
    ypredTest = np.loadtxt(
        os.path.join(folder_path, "ypred_PLSR_test_" + str(fold_idx) + ".csv"),
        delimiter=",",
    )
    yactTest = np.loadtxt(
        os.path.join(folder_path, "y_test_" + str(fold_idx) + ".csv"), delimiter=","
    )

    # scale the data
    scaler = StandardScaler()
    wavexTrain_scaled = scaler.fit_transform(wavexTrain)
    wavexTest_scaled = scaler.transform(wavexTest)

    return (
        wavexTrain_scaled,
        wavecoeffsTrain,
        ypredTrain,
        yactTrain,
        interceptTrain,
        wavexTest_scaled,
        ypredTest,
        yactTest,
    )

In [ ]:
def load_preprocessed_spectra():
    """
    Load RNV preprocessed spectral data and sample information.
    """
    base_path = "../ModelData/"
    data = {
        "wavenumbers": np.loadtxt(f"{base_path}RNV_wavenumbers.csv", delimiter=","),
        "intensities": np.loadtxt(f"{base_path}RNV_intensities.csv", delimiter=","),
        "samples": np.loadtxt(
            f"{base_path}samples.txt", delimiter=",", dtype=str
        ).tolist(),
        "labels": np.loadtxt(
            f"{base_path}labels.txt", delimiter=",", dtype=str
        ).tolist(),
        "numlabels": np.loadtxt(f"{base_path}numlabels.csv", delimiter=",", dtype=int),
    }

    with open(f"{base_path}RNV_column_names.txt", "r") as f:
        data["column_names"] = [line.strip() for line in f]

    # Ensure correct shape: samples should be in columns, not rows
    if data["intensities"].shape[0] == len(data["samples"]):
        data["intensities"] = data["intensities"].T

    return data

## Feature Selection: Use LASSO to reduce the number of candidates to a reasonable number

In [ ]:
def reducecandidates(initnum, wavex, wavecoeffs, ypred, alphamin, alphamax):
    """Run LASSO to get a minimum number of candidates."""

    numcoefflist = []
    rmselist = []
    rmselist2 = []
    allalpha = np.logspace(alphamin, alphamax, num=24)
    numcoeffbest = len(wavex[0, :])
    important_try = []

    print("log(alpha) numcoeff numcoeffbest initnum")

    for alpha in allalpha:

        clf = linear_model.Lasso(alpha=alpha, max_iter=40000)

        # run LASSO
        clf.fit(wavex, ypred)
        np.set_printoptions(suppress=True)
        numcoeff = int(np.sum(np.abs(clf.coef_) > 1e-10))
        numcoefflist.append(numcoeff)

        # compute RMSE for LASSO
        yimportant = clf.predict(wavex)
        rmse = np.sqrt(np.mean((yimportant - ypred) ** 2))
        rmselist.append(rmse)

        # refit a linear model keeping only non-zero LASSO coefficents
        important_guess = np.arange(len(wavex[0, :]))[(np.abs(clf.coef_) > 1e-10)]
        model = linear_model.LinearRegression()
        model.fit(wavex[:, np.array(important_guess).astype(int)], ypred)

        # compute RMSE for refit model
        yimportant2 = model.predict(wavex[:, np.array(important_guess).astype(int)])
        rmse2 = np.sqrt(np.mean((yimportant2 - ypred) ** 2))
        rmselist2.append(rmse2)

        print(np.log10(alpha), numcoeff, numcoeffbest, initnum)

        if numcoeff < numcoeffbest and numcoeff > initnum:
            important_try = np.arange(len(wavex[0, :]))[(np.abs(clf.coef_) > 1e-10)]
            numcoeffbest = numcoeff

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    ax1.set_xlabel("log(alpha)")
    ax1.plot(np.log10(allalpha), rmselist, "o", label="RMSE LASSO")
    ax1.plot(np.log10(allalpha), rmselist2, "s", label="RMSE refit")
    ax1.set_ylabel("RMSE")
    ax2.plot(np.log10(allalpha), numcoefflist, "rx", label="No. coeffs.")
    ax2.set_ylabel("number of coeff")

    ax1.legend()
    ax2.legend()
    plt.show()

    return important_try

## Brute Force Selection: Eliminate variables

In [ ]:
def eliminate_variables(startnum, finnum, important_try, wavex, ypred, tol):
    """Brute froce try all variable combinations."""

    important_trim = np.copy(important_try)

    rmselist = []
    keeplist = []

    for j in np.arange(startnum, finnum + 1):

        bestrmse = np.inf
        keep = np.array([])

        for i in itertools.combinations(important_try, j):

            test = np.array(i)

            model = linear_model.LinearRegression()
            model.fit(wavex[:, test.astype(int)], ypred)

            important_coeff = model.coef_
            important_intercept = model.intercept_

            yimportant = model.predict(wavex[:, test.astype(int)])
            rmse = np.sqrt(np.mean((yimportant - ypred) ** 2))

            if rmse < bestrmse:
                bestrmse = rmse
                keep = test

            if rmse < tol:
                rmselist.append(rmse)
                keeplist.append(test)

        print(j, bestrmse, keep)

        rmselist.append(bestrmse)
        keeplist.append(keep)

    return bestrmse, keep, rmselist, keeplist

## Compute ratios of RMSE

In [ ]:
def getratios(rmselist):
    """Compute ratios of RMSE at increasing number of important wavenumbers."""

    ratio = rmselist[:-1]
    for i in range(len(rmselist) - 1):
        ratio[i] = ratio[i] / rmselist[i + 1]

    return np.array(ratio)

## Linear SHAP

In [ ]:
def linearshap(coeffs, intercept, x):
    """Compute SHAP values analytically for a linear equation."""

    # determine predictions
    y = np.matmul(x, coeffs) + intercept

    # determine mean
    ymean = y.mean()

    # compute shap
    xx = np.transpose(np.transpose(x) - x.mean(axis=0).reshape(-1, 1))
    shapvalues = np.multiply(coeffs, xx)

    # base values
    shapbase = y * 0 + ymean

    shapex = shap.Explanation(values=shapvalues, base_values=shapbase, data=x)

    return shapex

## SHAP reducer

In [ ]:
def SHAPreducer(fullcoeffs, intercept, reducedcoeffs, keep, wavex, wavenumbermapper):
    """Given the important wavenumbers (keep), all wavenumbers, coefficents for both cases, create SHAP values."""

    from numpy.dtypes import StringDType

    # compute SHAP on everything
    coeffs = np.concatenate((reducedcoeffs, -1 * reducedcoeffs, fullcoeffs))
    data = np.concatenate(
        (
            wavex[:, np.array(keep).astype(int)],
            wavex[:, np.array(keep).astype(int)],
            wavex,
        ),
        axis=1,
    )

    shap_values = linearshap(coeffs, intercept, data)

    # create a mapper array to link a wavenumber to which SHAP will be computed.
    maxval = len(wavex[0])
    mapper = np.zeros(maxval + len(keep)) + maxval
    mapper = np.concatenate((keep, mapper.astype(int)))

    mapperunique = np.unique(mapper)

    # create the reduced SHAP
    shap_values_short = copy.deepcopy(shap_values)

    shap_values_short.values = np.zeros((len(wavex), len(mapperunique)))
    shap_values_short.data = np.zeros(
        (len(wavex), len(mapperunique))
    )  # This is wrong, but we don't actually use it.

    labels = np.array([""] * len(mapperunique)).astype(StringDType())
    labels[-1] = "Everything else"

    for i, mapval in enumerate(mapper):

        index = np.argwhere(mapperunique == mapval)[0][0]
        # print(i, mapval, index)
        shap_values_short.values[:, index] += shap_values.values[:, i]

        if mapval < maxval:
            labels[index] += str(wavenumbermapper[keep[i]]) + " "
            if mapval == keep[i]:
                shap_values_short.data[:, index] = shap_values.data[:, i]

    return shap_values_short, labels

## Correlation analysis

In [ ]:
def compute_correlation(
    important_indices,
    property_name,
    spectral_data,
    polymer_filter=None,
    split_by_polymer=False,
):
    """
    Compute correlations between important wavenumbers and all others.
    """
    intensities = spectral_data["intensities"]
    wavenumbers = spectral_data["wavenumbers"]
    column_names = spectral_data["column_names"]
    samples = spectral_data["samples"]
    numlabels = spectral_data["numlabels"]

    # assign numeric labels to each replicate
    replicate_labels = np.array(
        [
            numlabels[
                next(i for i, sample in enumerate(samples) if col.startswith(sample))
            ]
            for col in column_names
        ]
    )

    def compute_corr_df(subset, label_suffix=""):
        corr = np.corrcoef(subset, rowvar=False)
        df = pd.DataFrame(index=wavenumbers)
        for idx in important_indices:
            w = int(wavenumbers[idx])
            col_name = (
                f"{property_name}{label_suffix}corr_{w}"
                if label_suffix
                else f"Correlation_with_{w}"
            )
            df[col_name] = corr[idx, :]
        return df

    if split_by_polymer:
        polymer_groups = {"PE": [0, 1, 2], "PP": [3]}
        return {
            name: compute_corr_df(
                intensities[np.isin(replicate_labels, ids)], f"_{name}_"
            )
            for name, ids in polymer_groups.items()
        }
    else:
        if polymer_filter is not None:
            intensities = intensities[np.isin(replicate_labels, polymer_filter)]
        return compute_corr_df(intensities)

## Visualization

In [ ]:
def plot_correlation_scatter(
    correlation_df,
    wavenumbers,
    important_indices,
    shap_values,
    labels,
    property_name,
    polymer_type=None,
    threshold=0.9,
):
    """
    Plot scatter of wavenumbers with correlation > threshold for each important wavenumber.
    """

    def find_corr_col(w):
        col_variants = [
            f"Correlation_with_{int(w)}",
            f"{property_name}_{polymer_type}_corr_{int(w)}" if polymer_type else None,
            f"{property_name}_corr_{int(w)}",
        ]
        return next(
            (col for col in col_variants if col and col in correlation_df.columns), None
        )

    important_wavenumbers = wavenumbers[important_indices]
    n_important = len(important_indices)
    mean_abs_shap = np.mean(np.abs(shap_values.values[:, :n_important]), axis=0)
    order = np.argsort(mean_abs_shap)

    y_positions = np.linspace(-0.4, 0.3, len(order))
    plt.figure(figsize=(6, 3.4))
    sc = None

    for idx, y in zip(order, y_positions):
        w = important_wavenumbers[idx]
        col_name = find_corr_col(w)

        if col_name:
            corr = correlation_df[col_name]
            sig_mask = np.abs(corr) > threshold
            if sig_mask.any():
                sc = plt.scatter(
                    wavenumbers[sig_mask],
                    [y] * sig_mask.sum(),
                    c=corr[sig_mask].values,
                    cmap=shap.plots.colors.red_blue,
                    marker="|",
                    s=150,
                    vmin=-1,
                    vmax=1,
                )

    if sc:
        cbar = plt.colorbar(sc, label="Correlation coefficient")
        cbar.ax.tick_params(labelsize=9)
        cbar.set_label("Correlation coefficient", fontsize=10)

    plt.xlabel("Wavenumbers [cm$^{-1}$]", fontsize=10)
    plt.xlim(3500, 12500)
    plt.xticks(np.arange(4000, 12001, 2000), fontsize=10)
    plt.yticks(
        y_positions, [f"{int(w)}" for w in important_wavenumbers[order]], fontsize=10
    )
    plt.ylim(-1.0, 0.7)

    title = f"{property_name.capitalize()}"
    if polymer_type:
        title += f" - {polymer_type} samples"
    plt.title(title, fontsize=11)

    ymin, ymax = plt.ylim()
    plt.axvspan(5848, 5917, color="orange", alpha=0.2)
    plt.vlines([8390, 8427], ymin, ymax, color="orange", linewidth=0.5)
    plt.subplots_adjust(left=0.15, right=0.85, top=0.85, bottom=0.32)

## Make parity plots

In [ ]:
def parityplots(
    wavexTrain, wavexTest, keep, model, ypredTrain, ypredTest, yactTrain, yactTest
):
    """Create parity plots."""

    yxaiTrain = model.predict(wavexTrain[:, np.array(keep).astype(int)])
    yxaiTest = model.predict(wavexTest[:, np.array(keep).astype(int)])

    plt.plot(ypredTrain, yxaiTrain, "ro", label="train")
    plt.plot(ypredTest, yxaiTest, "bo", label="test")
    plt.plot(ypredTrain, ypredTrain)
    plt.ylabel("XAI predictions")
    plt.xlabel("PLSR predictions")
    plt.legend()
    plt.show()

    plt.plot(yactTrain, yxaiTrain, "ro", label="train")
    plt.plot(yactTest, yxaiTest, "bo", label="test")
    plt.plot(yactTrain, yactTrain)
    plt.ylabel("XAI predictions")
    plt.xlabel("Measured values")
    plt.legend()
    plt.show()

    return

# Get measurement values for connecting sample name to measured value

In [ ]:
file_path = "../Data/PropertyMeasurements.csv"

dfprop = pd.read_csv(file_path, usecols=[0, 1, 2, 3])

# remove units for ease of use
header = dfprop.columns.tolist()
header = [label.split()[0] for label in header]
dfprop.columns = header

dfprop.head()

In [ ]:
spectral_data = load_preprocessed_spectra()
RNV_q = spectral_data["wavenumbers"]

# Density

## Get the data

In [ ]:
(
    wavexTrain,
    wavecoeffsTrain,
    ypredTrain,
    yactTrain,
    interceptTrain,
    wavexTest,
    ypredTest,
    yactTest,
) = getdata("density")
rmse = np.sqrt(
    np.mean(
        ((np.matmul(wavexTrain, wavecoeffsTrain) + interceptTrain) - ypredTrain) ** 2.0
    )
)
print("RMSE between prediction and recalculated prediction (should be 0):", rmse)

## Reduce the number of potential candidates via LASSO

The number of variables keep should be increased until either important wavenumbers are unchanged or error is larger. In general, 30, 45, and 60 are used.

In [ ]:
important_try = reducecandidates(
    30, wavexTrain, wavecoeffsTrain, ypredTrain, -5.7, -2.5
)

## Brute force identify important wavenumbers

This is the most computatational expense step. Results are saved below, so it is commented out.

In [ ]:
# bestrmse, keep, rmselist, keeplist = eliminate_variables(1, 5, important_try, wavexTrain, ypredTrain, 2e-13)

### Print the rmselist and keeplist

In [ ]:
# print(rmselist)
# print([list(item) for item in keeplist])

### Save previously run results

In [ ]:
rmselist30 = [
    np.float64(0.00922178558089218),
    np.float64(0.005522627792975964),
    np.float64(0.0033134125349020744),
    np.float64(0.0025977091987638983),
    np.float64(0.0020865412794736837),
]
keeplist30 = [
    [np.int64(2395)],
    [np.int64(1643), np.int64(1658)],
    [np.int64(1016), np.int64(2367), np.int64(2391)],
    [np.int64(1658), np.int64(1992), np.int64(2367), np.int64(2391)],
    [np.int64(167), np.int64(691), np.int64(1015), np.int64(1642), np.int64(2395)],
]

In [ ]:
rmselist45 = [
    np.float64(0.00922178558089218),
    np.float64(0.005522627792975964),
    np.float64(0.0033134125349020744),
    np.float64(0.002540908953017799),
    np.float64(0.0021131752686593336),
]
keeplist45 = [
    [np.int64(2395)],
    [np.int64(1643), np.int64(1658)],
    [np.int64(1016), np.int64(2367), np.int64(2391)],
    [np.int64(694), np.int64(1015), np.int64(1643), np.int64(2391)],
    [np.int64(52), np.int64(691), np.int64(1015), np.int64(1642), np.int64(2395)],
]

In [ ]:
rmselist60 = [
    np.float64(0.00922178558089218),
    np.float64(0.005248959624292605),
    np.float64(0.0033134125349020744),
    np.float64(0.002540908953017799),
    np.float64(0.002122974657264155),
]
keeplist60 = [
    [np.int64(2395)],
    [np.int64(1645), np.int64(1658)],
    [np.int64(1016), np.int64(2367), np.int64(2391)],
    [np.int64(694), np.int64(1015), np.int64(1643), np.int64(2391)],
    [np.int64(46), np.int64(688), np.int64(1015), np.int64(1642), np.int64(2395)],
]

## Decide which list to use and how many important wavenumbers to keep

In [ ]:
plt.plot(np.arange(len(rmselist30)) + 1, rmselist30, ".", label="30")
plt.plot(np.arange(len(rmselist45)) + 1, rmselist45, "x", label="45")
plt.plot(np.arange(len(rmselist60)) + 1, rmselist60, "s", label="60")
plt.xlabel("Number of important wavenumbers")
plt.ylabel("RMSE")
plt.legend()
plt.show()

In [ ]:
print(getratios(rmselist30))

We see that the improvement drops after 3 important wavenumbers. The error for 3 wavenumbers is also around measurement error and lower than model error, so this is a good choice.

In [ ]:
imp = 3 - 1  # due to 0 indexing
print(keeplist30[imp])
print(keeplist45[imp])
print(keeplist60[imp])

The important wavenumbers are unaffected by the intial number selected by LASSO, so we can pick any of them.

In [ ]:
keep_density = keeplist30[2]
print(keep_density)

## Look at the correlations between important wavenumbers

In [ ]:
df = pd.DataFrame(wavexTrain[:, keep_density], columns=keep_density)

corr = df.corr()
corr.style.background_gradient(cmap="coolwarm")

## Train the model again and compute SHAP values

In [ ]:
model = linear_model.LinearRegression()
model.fit(wavexTrain[:, np.array(keep_density).astype(int)], ypredTrain)

important_coeff = model.coef_
important_intercept = model.intercept_

In [ ]:
shap_values, labels = SHAPreducer(
    wavecoeffsTrain,
    interceptTrain,
    model.coef_,
    keep_density,
    np.concatenate((wavexTrain, wavexTest)),
    RNV_q,
)

ax = shap.summary_plot(shap_values, shap_values.data, feature_names=labels, show=False)
plt.savefig("density.jpg")

## Create parity plots

In [ ]:
parityplots(
    wavexTrain,
    wavexTest,
    keep_density,
    model,
    ypredTrain,
    ypredTest,
    yactTrain,
    yactTest,
)

In [ ]:
for i, keep_idx in enumerate(keep_density):
    print(f"keep wavenumber: {RNV_q[keep_idx]}")

## Compute Pearson's correlation coefficients for important wavenumbers for sensity and plot key spectral regions with high coefficients

In [ ]:
correlation_df_density = compute_correlation(
    important_indices=keep_density,
    property_name="density",
    spectral_data=spectral_data,
    split_by_polymer=False,
)

# Plot key spectral regions
plot_correlation_scatter(
    correlation_df_density,
    RNV_q,
    keep_density,
    shap_values,
    labels,
    threshold=0.9,
    property_name="density",
)

# SCB

## Get the data

In [ ]:
(
    wavexTrain,
    wavecoeffsTrain,
    ypredTrain,
    yactTrain,
    interceptTrain,
    wavexTest,
    ypredTest,
    yactTest,
) = getdata("SCB/")
rmse = np.sqrt(
    np.mean(
        ((np.matmul(wavexTrain, wavecoeffsTrain) + interceptTrain) - ypredTrain) ** 2.0
    )
)
print("RMSE between prediction and recalculated prediction (should be 0):", rmse)

## Reduce the number of potential candidates via LASSO

In [ ]:
important_try = reducecandidates(30, wavexTrain, wavecoeffsTrain, ypredTrain, -3.5, 0)

## Brute force identify important wavenumbers

This is the most computatational expense step. Results are saved below, so it is commented out.

In [ ]:
# bestrmse, keep, rmselist, keeplist = eliminate_variables(1, 5, important_try, wavexTrain, ypredTrain, 2e-13)

### Print the rmselist and keeplist

In [ ]:
# print(rmselist)
# print([list(item) for item in keeplist])

In [ ]:
### Save previously run results

In [ ]:
rmselist30 = [
    np.float64(22.04513919618388),
    np.float64(10.9276408954596),
    np.float64(5.678916003605397),
    np.float64(4.584504402446514),
    np.float64(4.001900761640599),
]
keeplist30 = [
    [np.int64(2422)],
    [np.int64(2124), np.int64(2303)],
    [np.int64(1167), np.int64(1632), np.int64(2421)],
    [np.int64(1167), np.int64(1632), np.int64(2418), np.int64(3597)],
    [np.int64(1605), np.int64(1632), np.int64(1813), np.int64(2422), np.int64(3622)],
]

In [ ]:
rmselist45 = [
    np.float64(22.04513919618388),
    np.float64(10.9276408954596),
    np.float64(5.6830435160832256),
    np.float64(4.449087480242264),
    np.float64(3.924620614067766),
]
keeplist45 = [
    [np.int64(2422)],
    [np.int64(2124), np.int64(2303)],
    [np.int64(1166), np.int64(1632), np.int64(2421)],
    [np.int64(1166), np.int64(1626), np.int64(2410), np.int64(3019)],
    [np.int64(1602), np.int64(1623), np.int64(1821), np.int64(2421), np.int64(3025)],
]

In [ ]:
rmselist60 = [
    np.float64(22.04513919618388),
    np.float64(10.74381694850665),
    np.float64(5.81341318068204),
    np.float64(4.378641027407042),
]
keeplist60 = [
    [np.int64(2422)],
    [np.int64(2124), np.int64(2304)],
    [np.int64(1166), np.int64(1631), np.int64(2422)],
    [np.int64(1166), np.int64(1630), np.int64(2410), np.int64(3031)],
]

## Decide which list to use and how many important wavenumbers to keep

In [ ]:
plt.plot(np.arange(len(rmselist30)) + 1, rmselist30, ".", label="30")
plt.plot(np.arange(len(rmselist45)) + 1, rmselist45, "x", label="45")
plt.plot(np.arange(len(rmselist60)) + 1, rmselist60, "s", label="60")
plt.xlabel("Number of important wavenumbers")
plt.ylabel("RMSE")
plt.legend()
plt.show()

In [ ]:
print(getratios(rmselist30))

We see that the improvement drops after 3 important wavenumbers. The error for 3 wavenumbers is also around model and measurement error, so this is a good choice.

In [ ]:
imp = 3 - 1  # due to 0 indexing
print(keeplist30[imp])
print(keeplist45[imp])
print(keeplist60[imp])

All three are very close. Pick the lowest error.

In [ ]:
print(rmselist30[imp])
print(rmselist45[imp])
print(rmselist60[imp])

In [ ]:
keep_SCB = keeplist30[2]
print(keep_SCB)

## Look at the correlations between important wavenumbers

In [ ]:
df = pd.DataFrame(wavexTrain[:, keep_SCB], columns=keep_SCB)

corr = df.corr()
corr.style.background_gradient(cmap="coolwarm")

## Train the model again and compute SHAP values

In [ ]:
model = linear_model.LinearRegression()
model.fit(wavexTrain[:, np.array(keep_SCB).astype(int)], ypredTrain)

important_coeff = model.coef_
important_intercept = model.intercept_

In [ ]:
shap_values, labels = SHAPreducer(
    wavecoeffsTrain,
    interceptTrain,
    model.coef_,
    keep_SCB,
    np.concatenate((wavexTrain, wavexTest)),
    RNV_q,
)

ax = shap.summary_plot(shap_values, shap_values.data, feature_names=labels, show=False)
plt.savefig("SCB.jpg")

In [ ]:
keep_SCB

## Create parity plots

In [ ]:
parityplots(
    wavexTrain, wavexTest, keep_SCB, model, ypredTrain, ypredTest, yactTrain, yactTest
)

## Compute Pearson's correlation coefficients for important wavenumbers for SCB and plot key spectral regions with high coefficients

In [ ]:
correlation_df_SCB = compute_correlation(
    important_indices=keep_SCB,
    property_name="SCB",
    spectral_data=spectral_data,
    split_by_polymer=False,
)

In [ ]:
plot_correlation_scatter(
    correlation_df_SCB,
    RNV_q,
    keep_SCB,
    shap_values,
    labels,
    threshold=0.9,
    property_name="SCB",
)

# Crystallinity

## Get the data

In [ ]:
(
    wavexTrain,
    wavecoeffsTrain,
    ypredTrain,
    yactTrain,
    interceptTrain,
    wavexTest,
    ypredTest,
    yactTest,
) = getdata("Crystallinity/")
rmse = np.sqrt(
    np.mean(
        ((np.matmul(wavexTrain, wavecoeffsTrain) + interceptTrain) - ypredTrain) ** 2.0
    )
)
print("RMSE between prediction and recalculated prediction (should be 0):", rmse)

## Reduce the number of potential candidates via LASSO

The number of variables keep should be increased until either important wavenumbers are unchanged or error is larger. In general, 30, 45, and 60 are used.

In [ ]:
important_try = reducecandidates(60, wavexTrain, wavecoeffsTrain, ypredTrain, -4, -1.5)

## Brute force identify important wavenumbers

This is the most computatational expense step. Results are saved below, so it is commented out.

In [ ]:
# bestrmse, keep, rmselist, keeplist = eliminate_variables(1, 4, important_try, wavexTrain, ypredTrain, 2e-13)

### Print the rmselist and keeplist

In [ ]:
# print(rmselist)
# print([list(item) for item in keeplist])

### Save previously run results

In [ ]:
rmselist30 = [
    np.float64(0.08178873550099912),
    np.float64(0.05376756079953261),
    np.float64(0.0461775627409034),
    np.float64(0.038237664862284325),
    np.float64(0.033924346719240696),
]
keeplist30 = [
    [np.int64(1660)],
    [np.int64(1639), np.int64(1660)],
    [np.int64(1638), np.int64(1660), np.int64(2784)],
    [np.int64(1638), np.int64(1660), np.int64(2961), np.int64(3919)],
    [np.int64(1639), np.int64(1660), np.int64(2716), np.int64(2777), np.int64(3919)],
]

In [ ]:
rmselist45 = [
    np.float64(0.08178873550099912),
    np.float64(0.05376756079953261),
    np.float64(0.046205982030066206),
    np.float64(0.037393546151287156),
    np.float64(0.033924346719240696),
]
keeplist45 = [
    [np.int64(1660)],
    [np.int64(1639), np.int64(1660)],
    [np.int64(1638), np.int64(1660), np.int64(2777)],
    [np.int64(1638), np.int64(1660), np.int64(2961), np.int64(4011)],
    [np.int64(1639), np.int64(1660), np.int64(2716), np.int64(2777), np.int64(3919)],
]

In [ ]:
rmselist60 = [
    np.float64(0.08178873550099912),
    np.float64(0.05376756079953261),
    np.float64(0.04512420282064609),
    np.float64(0.037393546151287156),
]
keeplist60 = [
    [np.int64(1660)],
    [np.int64(1639), np.int64(1660)],
    [np.int64(1331), np.int64(1660), np.int64(2798)],
    [np.int64(1638), np.int64(1660), np.int64(2961), np.int64(4011)],
]

In [ ]:
rmselist75 = [
    np.float64(0.08178873550099912),
    np.float64(0.05376756079953261),
    np.float64(0.04512420282064609),
    np.float64(0.03817743231702813),
]
keeplist75 = [
    [np.int64(1660)],
    [np.int64(1639), np.int64(1660)],
    [np.int64(1331), np.int64(1660), np.int64(2798)],
    [np.int64(1639), np.int64(1660), np.int64(3763), np.int64(3919)],
]

In [ ]:
## Decide which list to use and how many important wavenumbers to keep

In [ ]:
plt.plot(np.arange(len(rmselist30)) + 1, rmselist30, ".", label="30")
plt.plot(np.arange(len(rmselist45)) + 1, rmselist45, "x", label="45")
plt.plot(np.arange(len(rmselist60)) + 1, rmselist60, "s", label="60")
plt.plot(np.arange(len(rmselist75)) + 1, rmselist75, "o", label="75")
plt.xlabel("Number of important wavenumbers")
plt.ylabel("RMSE")
plt.legend()
plt.show()

In [ ]:
print(getratios(rmselist30))

Improvement drops off after 2 wavenumbers, but the error is too high compared to experimental error. Keep one additional wavenumber.

In [ ]:
imp = 3 - 1  # due to 0 indexing
print(keeplist30[imp])
print(keeplist45[imp])
print(keeplist60[imp])
print(keeplist75[imp])

Once at least 60 is the initial selection for LASSO, the results are unchanged. Double check by looking at error.

In [ ]:
print(rmselist30[imp])
print(rmselist45[imp])
print(rmselist60[imp])
print(rmselist75[imp])

In [ ]:
keep_crystallinity = keeplist60[2]
print(keep_crystallinity)

## Look at the correlations between important wavenumbers

In [ ]:
df = pd.DataFrame(wavexTrain[:, keep_crystallinity], columns=keep_crystallinity)

corr = df.corr()
corr.style.background_gradient(cmap="coolwarm")

## Train the model again and compute SHAP values

In [ ]:
model = linear_model.LinearRegression()
model.fit(wavexTrain[:, np.array(keep_crystallinity).astype(int)], ypredTrain)

important_coeff = model.coef_
important_intercept = model.intercept_

In [ ]:
shap_values, labels = SHAPreducer(
    wavecoeffsTrain,
    interceptTrain,
    model.coef_,
    keep_crystallinity,
    np.concatenate((wavexTrain, wavexTest)),
    RNV_q,
)

ax = shap.summary_plot(shap_values, shap_values.data, feature_names=labels, show=False)
plt.savefig("crystallinity.jpg")

In [ ]:
keep_crystallinity

## Create parity plots

In [ ]:
parityplots(
    wavexTrain,
    wavexTest,
    keep_crystallinity,
    model,
    ypredTrain,
    ypredTest,
    yactTrain,
    yactTest,
)

In [ ]:
crystallinity_corr = compute_correlation(
    important_indices=keep_crystallinity,
    property_name="crystallinity",
    spectral_data=spectral_data,
    split_by_polymer=True,
)

# Get PE and PP correlation dataframes
correlation_df_crys_PE = crystallinity_corr["PE"]
correlation_df_crys_PP = crystallinity_corr["PP"]

## Compute Pearson's correlation coefficients for important wavenumbers for crystallinity and plot key spectral regions with high coefficients

In [ ]:
# For PE samples:
plot_correlation_scatter(
    correlation_df=correlation_df_crys_PE,
    wavenumbers=spectral_data["wavenumbers"],
    important_indices=keep_crystallinity,
    shap_values=shap_values,
    labels=labels,
    property_name="crystallinity",
    polymer_type="PE",
    threshold=0.9,
)

plt.savefig("crystallinity_PE_corr.png", dpi=300)
plt.show()

# For PP samples:
plot_correlation_scatter(
    correlation_df=correlation_df_crys_PP,
    wavenumbers=spectral_data["wavenumbers"],
    important_indices=keep_crystallinity,
    shap_values=shap_values,
    labels=labels,
    property_name="crystallinity",
    polymer_type="PP",
    threshold=0.9,
)

plt.savefig("crystallinity_PP_corr.png", dpi=300)
plt.show()